In [1]:
import gym
import numpy as np
import random
from collections import Counter, deque
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.nn.utils as nn_utils
import torch.nn.functional as F
import torch.optim as optim

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [2]:
# Hyperparameters
batch_size = 32

gamma = 0.99

eps_start=1.0
eps_decay = 0.995
eps_min = 0.1      # Minimal exploration rate (epsilon-greedy)

num_rounds = 500
num_episodes = 50
learning_limit = 100
replay_limit = 1000  # Number of steps until starting replay
weight_update = 1000 # Number of steps until updating the target weights


In [3]:
#create model
class Model(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(Model, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_shape[0]*input_shape[1], 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, n_actions)
            
        )

    def forward(self, x):
        # flatten the observation space Box to linear tensor
        tensor_array = torch.from_numpy(state)
        x_flat = torch.flatten(tensor_array).to(torch.float32)
        return self.net(x_flat)

In [4]:
class SchedulerEnv(gym.Env):

    def __init__(self):
        
        #starting parameters
        num_gps = 100
        num_slots = 32
        num_pre_booked = 750
        to_book = [2,1,2,2,1,1,1,3,3,1,2,1,3,2,1,1,2,1,3,2,3,2]
        num_to_book = len(to_book)
        agent_pos = [0,0]
        reward_decay = 0.95
        
        #set parameters for the day
        self.num_gps = num_gps
        self.num_slots = num_slots
        self.num_pre_booked = num_pre_booked
        self.to_book = to_book
        self.num_to_book = num_to_book
        self.diary_slots = num_gps*num_slots
        self.agent_pos = agent_pos
        self.reward_decay = reward_decay

        #set action space to move around the grid
        self.action_space = gym.spaces.Discrete(4) #up, down, left, right
        
        #set observation space 
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(self.num_slots, self.num_gps), dtype=np.int32)
   
    #creates daily diary for each gp, randomly populates prebooked appointments and resets parameters
    def reset(self):

        #creates zero filled dataframe with row per time slot and column per gp
        self.state = np.zeros((self.num_slots, self.num_gps),dtype=float)

        #randomly enters a 1 for each pre booked appointments
        pre_booked = self.num_pre_booked
        while pre_booked>0:
            pre_booked -= 1
            self.state[np.random.randint(self.num_slots), np.random.randint(self.num_gps)] = 1
            
        #randomly sets the agent start space
        self.agent_pos = [np.random.randint(self.num_slots), np.random.randint(self.num_gps)]

        #resets parameters for new episode
        self.done = False
        self.reward = 0
        self.appt_idx = 0
        self.decay_steps = 1
        
        #print('starting state', self.state.sum(), self.state)

        return self.state
    
    #calculates new position of the agent based on the action
    def move_agent(self, action):

        #set boundaries for the grid
        max_row = self.num_slots - 1
        max_col = self.num_gps - 1

        #setting new co-ordinates for the agent
        new_row = self.agent_pos[0]
        new_col = self.agent_pos[1]

        #calculate what the new position may be based on the action without going out the grid
        if action == 0:
            #print('up')
            new_row = max(self.agent_pos[0] - 1, 0)
        if action == 1:
            #print('down')
            new_row = min(self.agent_pos[0] + 1, max_row)
        if action == 2:
            #print('left')
            new_col = max(self.agent_pos[1] - 1, 0)
        if action == 3:
            #print('right')
            new_col = min(self.agent_pos[1] + 1, max_col)

        new_pos = [new_row, new_col]
        #print('new pos', new_pos)

        return new_pos

    #checks if we can look to book appointment starting here
    def check_bookable(self):
        return self.state[self.agent_pos[0], self.agent_pos[1]] == 0.0
    
    #action if we can't book the appointment
    def invalid_booking(self):
        #print('cant book')
        self.decay_steps += 1
        self.reward = -1
        
    #action if we can book the appointment
    def valid_booking(self):
        #print('go ahead and book')
        self.appt_idx += 1
        self.decay_steps = 1
        self.reward = 1
    
    #checks if the appointment fits
    def check_and_book(self):
        
        max_row = self.num_slots - 1
        cells_to_check = self.to_book[self.appt_idx]
        
        if cells_to_check==1:
            #print('good to check for single')
            if self.state[self.agent_pos[0], self.agent_pos[1]] == 0:
                self.state[self.agent_pos[0], self.agent_pos[1]] = 1
                self.valid_booking()
            else:
                #print('single taken')
                self.invalid_booking()

        if cells_to_check==2:
            #check we're not at the bottom of the grid
            if self.agent_pos[0]<max_row:
                #check the next cells is also 0.0
                #print('good to check for double')
                if self.state[self.agent_pos[0], self.agent_pos[1]] == 0 and \
                self.state[(self.agent_pos[0]+1), self.agent_pos[1]] == 0:
                    self.state[self.agent_pos[0], self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+1), self.agent_pos[1]] = 1
                    self.valid_booking()
                    self.agent_pos = [(self.agent_pos[0]+1), self.agent_pos[1]]
                    #print('after booking', self.agent_pos)
                else:
                    #print('double taken')
                    self.invalid_booking()
            else:
                #print('not for double')
                self.invalid_booking()
                
        if cells_to_check==3:
            #check we're not at the bottom of the grid
            if self.agent_pos[0]+1<max_row:
                #print('good to check for treble')
                if self.state[self.agent_pos[0], self.agent_pos[1]] == 0 and \
                self.state[(self.agent_pos[0]+1), self.agent_pos[1]] == 0 \
                 and self.state[(self.agent_pos[0]+2), self.agent_pos[1]] == 0:
                    self.state[self.agent_pos[0], self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+1), self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+2), self.agent_pos[1]] = 1
                    self.valid_booking()
                    self.agent_pos = [(self.agent_pos[0]+2), self.agent_pos[1]]
                else:
                    #print('treble taken')
                    self.invalid_booking()
            else:
                #print('not for treble')
                self.invalid_booking()
                
        if cells_to_check==4:
            #check we're not at the bottom of the grid
            if self.agent_pos[0]+2<max_row:
                #check the next cells is also 0.0
                #print('good for quad')
                if self.state[self.agent_pos[0], self.agent_pos[1]] == 0 and \
                self.state[(self.agent_pos[0]+1), self.agent_pos[1]] == 0 \
                 and self.state[(self.agent_pos[0]+2), self.agent_pos[1]] == 0 and \
                self.state[(self.agent_pos[0]+3), self.agent_pos[1]] == 0:
                    self.state[self.agent_pos[0], self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+1), self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+2), self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+3), self.agent_pos[1]] = 1
                    self.valid_booking()
                    self.agent_pos = [(self.agent_pos[0]+3), self.agent_pos[1]]
                else:
                    #print('quad taken')
                    self.invalid_booking()
            else:
                #print('not for quad')
                self.invalid_booking()

        next_state = self.state

        return next_state

    def step(self, action):

        #get new position of agent based on action
        new_agent_pos = self.move_agent(action)
        #print('new and old pos', new_agent_pos, self.agent_pos)
        
        #if the agent is stuck on an edge then move to a new position
        if new_agent_pos == self.agent_pos:
            self.agent_pos = [np.random.randint(self.num_slots), np.random.randint(self.num_gps)]
            #print('here1', self.agent_pos)
        else:
            self.agent_pos = new_agent_pos
            #print('here2', self.agent_pos)
        
        #print('trying to book', self.to_book, self.appt_idx)
        
        #check if it's possible to book then book
        if self.check_bookable():
            #print('checked here')
            self.state = self.check_and_book()
        else:
            #print('not bookable')
            self.invalid_booking()
        
        #work out if episode complete
        if self.appt_idx == len(self.to_book):
            #print('all booked')
            self.done = True
  
        #work out rewards
        #self.reward = (1 - (self.reward_decay**self.decay_steps))
        
        #print('step', self.decay_steps, self.reward)
        #print('end step')

        info = {}
        return self.state, self.reward, self.done, info

In [ ]:
#device = "cuda"
device = "cpu"

env = SchedulerEnv()

#start writing to tensorboard
writer = SummaryWriter(comment="Scheduler DQN")

#create the current network and target network
policy_model = Model(env.observation_space.shape, env.action_space.n).to(device)

target_model = Model(env.observation_space.shape, env.action_space.n).to(device)
target_model.load_state_dict(policy_model.state_dict())

optimizer = optim.Adam(policy_model.parameters(), lr=0.001, eps=1e-3)

          # Exploration rate    
replay_buffer = deque(maxlen=1000)

step_idx = 0
epsilon = eps_start

for i in range(num_rounds):
    #change this for while not true once it works
    episode_reward = 0

    for j in range(num_episodes):
        
        step_idx += 1
        #print(i,j,step_idx)
        state = env.reset()

        #epsilon for epsilon greedy strategy  
        if epsilon > eps_min:
            epsilon *= eps_decay
            
        #print('epsilon', epsilon)
            
        check = policy_model(state)
        #print(check)

        # Select and perform an action
        if step_idx > learning_limit:
            if np.random.rand() > epsilon:
                action = torch.argmax(policy_model(state))
        else:
            action = np.random.randint(env.action_space.n)

        next_state, reward, done, _ = env.step(action)
        reward = torch.tensor([reward], device=device)
        episode_reward += reward
        #print('here rewards', episode_reward, reward, step_idx)

        # Store other info in replay buffer
        replay_buffer.append((state, action, reward, next_state, done))
        #print('len buffer', len(replay_buffer))

        # Move to the next state
        state = next_state

        if done:
            break
            
        #print('stopped episode', j)
        
    writer.add_scalar('episode_reward', episode_reward, step_idx)

    #once we're ready to learn then start learning with mini batches
    if len(replay_buffer) == replay_limit:
        print('replay buffer')
        minibatch = random.sample(replay_buffer, batch_size)

        for state, action, reward, next_state, done in minibatch:    
            optimizer.zero_grad()
            #pass state to policy to get qval from policy
            pred_qval = policy_model(state)

            #pass next state to target policy to get next set of qvals (future gains)
            if not done:
                next_qval = (reward + (gamma * max(target_model(next_state)))).detach()
                #next_qval = next_qval.detach()
            else:
                next_qval = reward    

            loss = F.mse_loss(pred_qval, next_qval)
            print('loss', loss)
            loss.backward()

            optimizer.step()
    print('step', step_idx)

    # Update the target network, copying all weights and biases in DQN
    # Periodically update the target network by Q network to target Q network
    if step_idx % weight_update == 0:
        print('update weights')
        # Update weights of target
        target_model.load_state_dict(policy_model.state_dict())

writer.close()

len buffer 1
len buffer 2
len buffer 3
len buffer 4
len buffer 5
len buffer 6
len buffer 7
len buffer 8
len buffer 9
len buffer 10
len buffer 11
len buffer 12
len buffer 13
len buffer 14
len buffer 15
len buffer 16
len buffer 17
len buffer 18
len buffer 19
len buffer 20
len buffer 21
len buffer 22
len buffer 23
len buffer 24
len buffer 25
len buffer 26
len buffer 27
len buffer 28
len buffer 29
len buffer 30
len buffer 31
len buffer 32
len buffer 33
len buffer 34
len buffer 35
len buffer 36
len buffer 37
len buffer 38
len buffer 39
len buffer 40
len buffer 41
len buffer 42
len buffer 43
len buffer 44
len buffer 45
len buffer 46
len buffer 47
len buffer 48
len buffer 49
len buffer 50
step 50
len buffer 51
len buffer 52
len buffer 53
len buffer 54
len buffer 55
len buffer 56
len buffer 57
len buffer 58
len buffer 59
len buffer 60
len buffer 61
len buffer 62
len buffer 63
len buffer 64
len buffer 65
len buffer 66
len buffer 67
len buffer 68
len buffer 69
len buffer 70
len buffer 71
len buf

len buffer 554
len buffer 555
len buffer 556
len buffer 557
len buffer 558
len buffer 559
len buffer 560
len buffer 561
len buffer 562
len buffer 563
len buffer 564
len buffer 565
len buffer 566
len buffer 567
len buffer 568
len buffer 569
len buffer 570
len buffer 571
len buffer 572
len buffer 573
len buffer 574
len buffer 575
len buffer 576
len buffer 577
len buffer 578
len buffer 579
len buffer 580
len buffer 581
len buffer 582
len buffer 583
len buffer 584
len buffer 585
len buffer 586
len buffer 587
len buffer 588
len buffer 589
len buffer 590
len buffer 591
len buffer 592
len buffer 593
len buffer 594
len buffer 595
len buffer 596
len buffer 597
len buffer 598
len buffer 599
len buffer 600
step 600
len buffer 601
len buffer 602
len buffer 603
len buffer 604
len buffer 605
len buffer 606
len buffer 607
len buffer 608
len buffer 609
len buffer 610
len buffer 611
len buffer 612
len buffer 613
len buffer 614
len buffer 615
len buffer 616
len buffer 617
len buffer 618
len buffer 619
l

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:85: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
replay buffer
loss tensor(1.0786, grad_fn=<MseLossBackward>)
loss tensor(0.9298, grad_fn=<MseLossBackward>)
loss tensor(1.0779, grad_fn=<MseLossBackward>)
loss tensor(1.0860, grad_fn=<MseLossBackward>)
loss tensor(1.1322, grad_fn=<MseLossBackward>)
loss tensor(1.0187, grad_fn=<MseLossBackward>)


len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
replay buffer
loss tensor(0.5293, grad_fn=<MseLossBackward>)
loss tensor(1.6880, grad_fn=<MseLossBackward>)
loss tensor(0.3908, grad_fn=<MseLossBackward>)
loss tensor(0.4981, grad_fn=<MseLossBackward>)
loss tensor(0.4577, grad_fn=<MseLossBackward>)
loss tensor(0.4865, grad_fn=<MseLossBackward>)
loss tensor(1.89

...

loss tensor(0.1889, grad_fn=<MseLossBackward>)
loss tensor(2.8079, grad_fn=<MseLossBackward>)
loss tensor(3.3299, grad_fn=<MseLossBackward>)
loss tensor(0.1328, grad_fn=<MseLossBackward>)
loss tensor(2.5174, grad_fn=<MseLossBackward>)
loss tensor(0.1969, grad_fn=<MseLossBackward>)
loss tensor(2.9552, grad_fn=<MseLossBackward>)
loss tensor(1.7203, grad_fn=<MseLossBackward>)
loss tensor(2.2700, grad_fn=<MseLossBackward>)
loss tensor(0.1638, grad_fn=<MseLossBackward>)
loss tensor(0.3791, grad_fn=<MseLossBackward>)
loss tensor(1.6422, grad_fn=<MseLossBackward>)
loss tensor(0.3912, grad_fn=<MseLossBackward>)
loss tensor(0.4734, grad_fn=<MseLossBackward>)
loss tensor(0.6719, grad_fn=<MseLossBackward>)
loss tensor(1.3315, grad_fn=<MseLossBackward>)
step 13150
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 10

len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
replay buffer
loss tensor(0.9041, grad_fn=<MseLossBackward>)
loss tensor(0.9484, grad_fn=<MseLossBackward>)
loss tensor(1.2448, grad_fn=<MseLossBackward>)
loss tensor(0.7524, grad_fn=<MseLossBackward>)
loss tensor(1.0548, grad_fn=<MseLossBackward>)
loss tensor(1.2404, grad_fn=<MseLossBackward>)
loss tensor(0.8980, grad_fn=<MseLossBackward>)
loss tensor(1.1652, grad_fn=<MseLossBackward>)
loss tensor(0.9054, grad_fn=<MseLossBackward>)
loss tensor(0.9344, grad_fn=<MseLossBackward>)
loss tensor(1.2592, grad_fn=<MseLossBackward>)
loss tensor(0.8931, grad_fn=<MseLossBackward>)
loss tensor(0.7752, grad_fn=<MseLossBackward>)
loss tensor(0.2756, grad_fn=<MseLossBackward>)
loss tensor(0.4815, grad_fn=<MseLossBackward>)
loss tens

len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
replay buffer
loss tensor(1.0396, grad_fn=<MseLossBackward>)
loss tensor(1.1347, grad_fn=<MseLossBackward>)
loss tensor(1.6253, grad_fn=<MseLossBackward>)
loss tensor(1.2703, grad_fn=<MseLossBackward>)
loss tensor(0.9340, grad_fn=<MseLossBackward>)
loss tensor(0.9521, grad_fn=<MseLossBackward>)
loss tensor(1.10

len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
replay buffer
loss tensor(0.3145, grad_fn=<MseLossBackward>)
loss tensor(0.8546, grad_fn=<MseLossBackward>)
loss tensor(0.5815, grad_fn=<MseLossBackward>)
loss tensor(0.6605, grad_fn=<MseLossBackward>)
loss tensor(0.6276, grad_fn=<MseLossBackward>)
loss tensor(0.8419, grad_fn=<MseLossBackward>)
loss tensor(0.65

len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
len buffer 1000
replay buffer
loss tensor(0.7240, grad_fn=<MseLossBackward>)
loss tensor(0.1116, grad_fn=<MseLossBackward>)
loss tensor(1.8190, grad_fn=<MseLossBackward>)
loss tensor(2.6523, grad_fn=<MseLossBackward>)
loss tensor(0.4355, grad_fn=<MseLossBackward>)
loss tensor(0.8163, grad_fn=<Ms

In [ ]:
check

In [ ]:
torch.argmax(check)

In [ ]:
np.random.randint(5, size=(2, 4))